In [397]:
import os
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import warnings
%matplotlib inline

rc('font', family='AppleGothic')
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

warnings.filterwarnings(action='ignore')
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [398]:
class Comparison:
    def __init__(self, df):
        self.df = df
        self.df_T = self.df.transpose()
        
    def ls(self, weather, order):
        if order == 'plus':
            return self.df_T[self.df_T[weather] > 0].index.to_list()
        elif order == 'minus':
            return self.df_T[self.df_T[weather] < 0].index.to_list()
    
    def power(self, weather, order ='plus'): # 음/양에 따라 내림차순 정렬
        plus = self.df_T[self.df_T[weather] > 0].index.to_list()
        minus = self.df_T[self.df_T[weather] < 0].index.to_list()
        
        if order == 'plus':
            return self.df_T[self.df.columns.isin(plus)].sort_values(by=weather, ascending=False)
        elif order == 'minus':
            return self.df_T[self.df.columns.isin(minus)].sort_values(by=weather, ascending=True)
        elif order == 'neutral':
            return self.df_T.reindex(self.df_T[weather].abs().sort_values(ascending=False).index)
        
    def big(self, cat, weather, order ='plus'): # 대분류 필터링 + 음/양에 따라 내림차순 정렬
        if cat == '가전':
            cat = '냉난방가전'
            
        reset = self.df_T.reset_index().rename(columns={'index':'sm_cat'})
        merged = pd.merge(reset, big_cat, how='outer', on='sm_cat')
        by_cat = merged[merged['big_cat']==cat]
        
        plus = by_cat[by_cat[weather] > 0]['sm_cat'].to_list()
        minus = by_cat[by_cat[weather] < 0]['sm_cat'].to_list()

        if order == 'plus':
            return by_cat[by_cat['sm_cat'].isin(plus)].sort_values(by=weather, ascending=False)
        elif order == 'minus':
            return by_cat[by_cat['sm_cat'].isin(minus)].sort_values(by=weather, ascending=True)
        elif order == 'neutral':
            return by_cat.reindex(by_cat[weather].abs().sort_values(ascending=False).index)

In [399]:
big_cat = pd.read_csv('./대분류.csv', index_col=0)
data = pd.read_csv('./target.csv', index_col=0)

products = data.columns[3:].to_list() # 종속 변수
big_cat = big_cat[big_cat['sm_cat'].isin(products)]

In [400]:
score_whole = pd.read_csv('./result_전기간 모델링 결과.csv', index_col=0)
score_warm = pd.read_csv('./result_따뜻한 달 모델링 결과.csv', index_col=0)
score_cool = pd.read_csv('./result_시원한 달 모델링 결과.csv', index_col=0)
score_rainy = pd.read_csv('./result_장마철 모델링 결과.csv', index_col=0)
score_2018 = pd.read_csv('./result_2018년 모델링 결과.csv', index_col=0)
score_2019 = pd.read_csv('./result_2019년 모델링 결과.csv', index_col=0)

In [401]:
whole = Comparison(score_whole)

warm = Comparison(score_warm)
cool = Comparison(score_cool)

rainy = Comparison(score_rainy)

year2018 = Comparison(score_2018)
year2019 = Comparison(score_2019)

dusty = Comparison(score_dusty)

전기간 (whole)
- 평균기온 민감(음/양)
- 일강수량 민감(음/양)
- 미세먼지 민감(음/양)
- 진짜 민감한 상품들


따뜻한 달(warm), 시원한 달(cool)
- 평균기온 민감(음/양)
- ~일강수량 민감(음/양)~
- 미세먼지 민감(음/양)
- 진짜 민감한 상품들

장마기간(rainy)
- ~평균기온 민감(음/양)~
- 일강수량 민감(음/양)
- ~미세먼지 민감(음/양)~
- 진짜 민감한 상품들

2018년, 2019년
- 평균기온 민감(음/양)
- 일강수량 민감(음/양)
- 미세먼지 민감(음/양)
- 진짜 민감한 상품들

# 1. 전기간

## 1.1. 평균기온

### 양의 상관관계

In [520]:
whole.big('뷰티', '평균기온', 'plus').head(10)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
19,바디케어용 제모제,1.1597,NaN,0.2520,0.7429,0.5378,뷰티
89,뷰티 타투,1.0996,NaN,0.0524,0.6988,0.8122,뷰티
142,애프터선,1.0780,NaN,0.1703,0.7386,0.8765,뷰티
97,선케어용 선밤,0.8507,-0.1482,0.1042,0.5373,0.5964,뷰티
25,스킨케어 코팩,0.8454,NaN,0.2670,0.7334,1.0962,뷰티
140,핸드워시/스크럽,0.8436,0.1766,0.5296,0.7334,0.6104,뷰티
29,데오드란트,0.7037,-0.0867,0.1688,0.6859,0.5558,뷰티
80,네일관리 소품,0.6413,0.1192,0.5477,0.7624,1.0831,뷰티
16,바디 클렌져,0.5937,0.1056,0.2695,0.7600,0.7649,뷰티
40,선크림,0.5857,NaN,0.2714,0.6696,0.5210,뷰티


In [521]:
whole.big('가전', '평균기온', 'plus').head(10)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
2,벽걸이형 선풍기,1.1119,-0.1647,NaN,0.5824,0.5311,냉난방가전
34,에어컨 리모컨,0.8871,-0.1185,-0.0681,0.5484,0.4008,냉난방가전
77,업소용 선풍기,0.8378,-0.1650,-0.0814,0.4895,0.5466,냉난방가전
107,탁상/USB 선풍기,0.7939,-0.1274,NaN,0.5560,0.5168,냉난방가전
151,이동형 에어컨,0.6776,-0.1566,-0.0533,0.4453,0.3816,냉난방가전
120,휴대용 선풍기,0.6302,-0.1494,-0.0469,0.5213,0.4710,냉난방가전
5,벽걸이 에어컨,0.6295,-0.1648,NaN,0.4681,0.3457,냉난방가전
79,냉풍기,0.5330,-0.1638,-0.0542,0.3157,0.3157,냉난방가전
35,스탠드형 에어컨,0.4763,-0.1417,0.1176,0.4297,0.4220,냉난방가전
111,멀티형 에어컨,0.3920,-0.1749,0.1123,0.3112,0.9030,냉난방가전


In [622]:
warm.big('가전', '평균기온', 'plus').head(10)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
34,에어컨 리모컨,1.1971,-0.1979,-0.2520,0.6181,0.4843,냉난방가전
2,벽걸이형 선풍기,1.1430,-0.1848,NaN,0.6333,0.5769,냉난방가전
77,업소용 선풍기,1.1012,-0.2628,-0.2232,0.5514,0.6307,냉난방가전
151,이동형 에어컨,0.8922,-0.2469,-0.1680,0.4979,0.4345,냉난방가전
107,탁상/USB 선풍기,0.7579,NaN,NaN,0.6083,0.5595,냉난방가전
79,냉풍기,0.7560,-0.2734,-0.1838,0.3671,0.3614,냉난방가전
120,휴대용 선풍기,0.7474,-0.2119,-0.0803,0.5811,0.5363,냉난방가전
5,벽걸이 에어컨,0.5293,-0.1803,0.1746,0.5276,0.3885,냉난방가전
61,전기온수기,0.4741,NaN,0.4917,0.6063,1.3891,냉난방가전
92,제습기,0.4048,0.3193,NaN,0.4690,0.4201,냉난방가전


In [626]:
cool.big('뷰티', '평균기온', 'plus').head(30)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
16,바디 클렌져,1.4441,NaN,0.7490,0.7362,0.7071,뷰티
140,핸드워시/스크럽,1.3581,NaN,0.7393,0.7179,0.6689,뷰티
89,뷰티 타투,1.2876,NaN,0.1037,0.5332,0.5490,뷰티
29,데오드란트,0.8735,NaN,0.6153,0.7454,0.7557,뷰티
40,선크림,0.8716,NaN,0.7697,0.7110,0.6917,뷰티
97,선케어용 선밤,0.7929,NaN,0.2079,0.4060,0.6202,뷰티
124,선로션,0.7480,NaN,0.4018,0.6752,1.0006,뷰티
19,바디케어용 제모제,0.6484,NaN,0.5946,0.6653,0.8202,뷰티
68,색조 메이크업 립밤,0.5400,NaN,0.5361,0.6511,0.5864,뷰티
130,기초 화장용 오일/앰플,0.5315,NaN,0.2838,0.5729,1.1818,뷰티


In [522]:
whole.big('식품', '평균기온', 'plus').head(10)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
101,참외/메론/수박,1.3078,-0.1310,NaN,0.6175,0.3171,식품
36,이온음료,1.2087,-0.1023,0.2747,0.8161,0.6847,식품
24,계란,1.1399,0.2328,0.4261,0.8032,0.6969,식품
54,다이어트용 헬스보충식품,1.1312,NaN,0.4745,0.7947,0.7003,식품
74,김치류,1.0635,0.1108,0.3165,0.7650,1.1114,식품
87,탄산수,1.0463,NaN,0.4839,0.8126,0.7090,식품
48,무김치,1.0151,NaN,0.1755,0.7658,0.9992,식품
22,음용 식초,0.9984,0.1170,0.3662,0.8097,0.9091,식품
106,돼지 곱창,0.9530,0.4047,0.6700,0.7201,0.3578,식품
75,아이스티,0.9450,NaN,0.2274,0.7929,0.8553,식품


###  음의 상관관계

In [404]:
whole.big('뷰티', '평균기온', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
10,핸드크림,-0.2227,0.2719,0.6853,0.6403,0.5495,뷰티
68,색조 메이크업 립밤,-0.1557,0.1905,0.5738,0.6120,0.5136,뷰티


In [629]:
whole.big('가전', '평균기온', 'minus').head(20)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
64,초음파식 가습기,-0.6073,0.2115,0.6885,0.4635,0.3353,냉난방가전
51,복합식 가습기,-0.4119,0.1535,0.5062,0.4524,0.6286,냉난방가전
150,컨벡터,-0.2802,0.1100,0.2941,0.2471,0.4743,냉난방가전
50,히터,-0.2655,NaN,0.2948,0.2731,0.3543,냉난방가전
144,라디에이터,-0.2633,0.0971,0.2703,0.2663,0.4198,냉난방가전
143,온풍기,-0.2403,0.0897,0.2648,0.2739,0.3436,냉난방가전
44,가열식 가습기,-0.2370,NaN,0.2917,0.3630,0.4308,냉난방가전
145,에어워셔,-0.1495,NaN,0.2618,0.5287,1.1214,냉난방가전
94,공기청정기,-0.1215,NaN,0.2149,0.5013,0.4889,냉난방가전
62,자연식 가습기,-0.1139,NaN,0.1985,0.3625,0.7413,냉난방가전


In [633]:
cool.big('가전', '평균기온', 'minus').head(20)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [628]:
warm.big('식품', '평균기온', 'minus').head(20)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
41,포도/거봉/체리,-0.2166,0.1559,0.6589,0.5117,0.8671,식품


In [406]:
whole.big('식품', '평균기온', 'minus').head(20)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
127,굴 생물,-0.4038,0.1416,0.4098,0.3833,0.3451,식품
122,감귤/한라봉/오렌지,-0.3862,0.2283,0.6075,0.6566,0.5533,식품
110,해초류,-0.1867,0.1201,0.2587,0.3896,1.3532,식품
82,유자차,-0.1848,0.2705,0.6637,0.5617,0.4989,식품
90,율무차,-0.1442,0.2462,0.6419,0.6374,0.7228,식품
46,코코아/핫초코,-0.0798,0.2292,0.5216,0.6827,0.7124,식품
21,랍스타,-0.0796,NaN,0.1195,0.1489,1.4973,식품


In [627]:
cool.big('식품', '평균기온', 'minus').head(20)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
38,딸기/복분자/블루베리,-1.4524,0.4873,1.1375,0.7332,0.7362,식품
65,유제품 음료,-0.7715,0.3691,0.4803,0.4027,0.2276,식품
110,해초류,-0.4620,0.1925,0.2879,0.3679,1.4523,식품
11,전복 생물,-0.2633,NaN,0.2658,0.3686,0.8558,식품
76,인삼/수삼/산삼,-0.2324,NaN,0.3956,0.5253,0.6907,식품


## 1.2. 일강수량

### 양의 상관관계

In [643]:
rainy.big('가전', '일강수량', 'plus').head(10)[['sm_cat', '일강수량']]

,sm_cat,일강수량
92,제습기,0.6351
103,공기정화 용품,0.4765
143,온풍기,0.3534


In [549]:
cool.big('뷰티', '일강수량', 'plus').head(10)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
116,뷰티용 뷰러,-0.5159,0.4502,0.9512,0.7516,0.9436,뷰티
80,네일관리 소품,NaN,0.2886,0.9118,0.7576,1.0585,뷰티
67,클렌징 티슈,0.4269,0.2361,0.4717,0.6880,0.9359,뷰티
8,바디케어용 때비누,NaN,0.2281,0.2782,0.4962,1.5571,뷰티
25,스킨케어 코팩,0.4482,0.1660,0.3266,0.6140,1.5377,뷰티


In [556]:
whole.big('식품', '일강수량', 'plus').head(10)[['sm_cat', '일강수량']]

,sm_cat,일강수량
106,돼지 곱창,0.4047
38,딸기/복분자/블루베리,0.2881
104,시금치,0.2878
26,둥굴레차,0.2848
123,곡물차,0.2795
82,유자차,0.2705
90,율무차,0.2462
24,계란,0.2328
138,생닭/닭부분육,0.2300
46,코코아/핫초코,0.2292


In [558]:
warm.big('식품', '일강수량', 'plus').head(10)[['sm_cat', '일강수량']]

,sm_cat,일강수량
46,코코아/핫초코,0.4727
90,율무차,0.3619
122,감귤/한라봉/오렌지,0.3096
4,초코우유,0.2841
102,견과류 밤,0.2760
65,유제품 음료,0.2701
26,둥굴레차,0.2668
106,돼지 곱창,0.2622
148,건대추,0.2451
138,생닭/닭부분육,0.2435


In [559]:
cool.big('식품', '일강수량', 'plus').head(10)[['sm_cat', '일강수량']]

,sm_cat,일강수량
38,딸기/복분자/블루베리,0.4873
104,시금치,0.4856
126,식혜/수정과,0.4734
45,탄산음료,0.4627
22,음용 식초,0.3778
65,유제품 음료,0.3691
125,꽃게,0.3445
30,파/양파,0.3332
106,돼지 곱창,0.3140
119,감자,0.2905


In [560]:
rainy.big('식품', '일강수량', 'plus').head(10)[['sm_cat', '일강수량']]

,sm_cat,일강수량
32,낙지,0.4552


### 음의 상관관계

In [639]:
whole.big('뷰티', '일강수량', 'minus').head(5)[['sm_cat', '일강수량']]

,sm_cat,일강수량
97,선케어용 선밤,-0.1482
29,데오드란트,-0.0867


In [638]:
whole.big('가전', '일강수량', 'minus').head(30)[['sm_cat', '일강수량']]

,sm_cat,일강수량
111,멀티형 에어컨,-0.1749
77,업소용 선풍기,-0.1650
5,벽걸이 에어컨,-0.1648
2,벽걸이형 선풍기,-0.1647
79,냉풍기,-0.1638
151,이동형 에어컨,-0.1566
120,휴대용 선풍기,-0.1494
35,스탠드형 에어컨,-0.1417
107,탁상/USB 선풍기,-0.1274
34,에어컨 리모컨,-0.1185


In [636]:
cool.big('가전', '일강수량', 'plus').head(20)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
92,제습기,0.9648,0.5528,0.5019,0.7339,1.1960,냉난방가전
152,의류건조기,NaN,0.2215,0.5760,0.6696,1.4645,냉난방가전


In [637]:
whole.big('식품', '일강수량', 'minus').head(5)[['sm_cat', '일강수량']]

,sm_cat,일강수량
101,참외/메론/수박,-0.1310
36,이온음료,-0.1023


## 1.3. 미세먼지

### 양의 상관관계

In [579]:
whole.big('식품', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
38,딸기/복분자/블루베리,NaN,0.2881,0.9880,0.7387,0.6283,식품
26,둥굴레차,NaN,0.2848,0.9225,0.7001,0.9418,식품
104,시금치,NaN,0.2878,0.7814,0.7269,0.7725,식품
123,곡물차,0.2769,0.2795,0.6947,0.6299,0.4726,식품
106,돼지 곱창,0.9530,0.4047,0.6700,0.7201,0.3578,식품


In [582]:
cool.big('식품', '미세먼지', 'plus').head(20)[['sm_cat', '미세먼지']]

,sm_cat,미세먼지
38,딸기/복분자/블루베리,1.1375
104,시금치,1.0005
6,생수,0.9902
30,파/양파,0.9768
126,식혜/수정과,0.9520
133,고추/피망/파프리카,0.9400
26,둥굴레차,0.9381
45,탄산음료,0.8696
24,계란,0.7814
119,감자,0.7246


In [577]:
warm.big('식품', '미세먼지', 'plus').head(30)[['sm_cat', '미세먼지']]

,sm_cat,미세먼지
26,둥굴레차,0.9211
46,코코아/핫초코,0.9206
90,율무차,0.9178
65,유제품 음료,0.8539
4,초코우유,0.8369
45,탄산음료,0.8120
122,감귤/한라봉/오렌지,0.7744
104,시금치,0.7635
119,감자,0.7374
38,딸기/복분자/블루베리,0.7271


In [578]:
whole.big('식품', '미세먼지', 'plus').head(20)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
38,딸기/복분자/블루베리,NaN,0.2881,0.9880,0.7387,0.6283,식품
26,둥굴레차,NaN,0.2848,0.9225,0.7001,0.9418,식품
104,시금치,NaN,0.2878,0.7814,0.7269,0.7725,식품
123,곡물차,0.2769,0.2795,0.6947,0.6299,0.4726,식품
106,돼지 곱창,0.9530,0.4047,0.6700,0.7201,0.3578,식품
82,유자차,-0.1848,0.2705,0.6637,0.5617,0.4989,식품
90,율무차,-0.1442,0.2462,0.6419,0.6374,0.7228,식품
122,감귤/한라봉/오렌지,-0.3862,0.2283,0.6075,0.6566,0.5533,식품
45,탄산음료,0.5859,0.1277,0.5802,0.7636,0.4396,식품
119,감자,0.6770,0.1252,0.5228,0.7507,0.5526,식품


In [416]:
whole.big('식품', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
38,딸기/복분자/블루베리,NaN,0.2881,0.9880,0.7387,0.6283,식품
26,둥굴레차,NaN,0.2848,0.9225,0.7001,0.9418,식품
104,시금치,NaN,0.2878,0.7814,0.7269,0.7725,식품
123,곡물차,0.2769,0.2795,0.6947,0.6299,0.4726,식품
106,돼지 곱창,0.9530,0.4047,0.6700,0.7201,0.3578,식품


### 음의 상관관계

In [417]:
whole.big('뷰티', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [418]:
whole.big('가전', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
77,업소용 선풍기,0.8378,-0.1650,-0.0814,0.4895,0.5466,냉난방가전
34,에어컨 리모컨,0.8871,-0.1185,-0.0681,0.5484,0.4008,냉난방가전
79,냉풍기,0.5330,-0.1638,-0.0542,0.3157,0.3157,냉난방가전
151,이동형 에어컨,0.6776,-0.1566,-0.0533,0.4453,0.3816,냉난방가전
120,휴대용 선풍기,0.6302,-0.1494,-0.0469,0.5213,0.4710,냉난방가전


In [419]:
whole.big('식품', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
3,옥수수,0.8502,NaN,-0.0447,0.5356,0.3203,식품


#  2. 따뜻한 달 / 시원한 달

## 2.1. 따뜻한 달 (4 ~ 9월)

### 2.1.1. 평균기온

### 양의 상관관계

In [528]:
warm.big('뷰티', '평균기온', 'plus').head(10)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
58,네일 메이크업 용품,1.1899,NaN,0.3651,0.7768,0.9121,뷰티
89,뷰티 타투,1.0691,NaN,NaN,0.7148,0.8897,뷰티
142,애프터선,1.0244,NaN,0.2922,0.7734,0.9777,뷰티
98,네일리무버,0.9144,0.1558,0.3814,0.8026,1.2453,뷰티
19,바디케어용 제모제,0.8873,NaN,0.5718,0.8005,0.6809,뷰티
140,핸드워시/스크럽,0.8784,NaN,0.4483,0.7268,0.5781,뷰티
25,스킨케어 코팩,0.8522,NaN,0.2988,0.7833,0.8868,뷰티
109,헤어스타일링용 흑채,0.7946,NaN,0.6455,0.7725,1.2472,뷰티
80,네일관리 소품,0.7503,NaN,0.4942,0.7622,1.1768,뷰티
130,기초 화장용 오일/앰플,0.6773,NaN,0.2672,0.6671,1.0025,뷰티


In [523]:
warm.big('가전', '평균기온', 'plus').head(10)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
34,에어컨 리모컨,1.1971,-0.1979,-0.2520,0.6181,0.4843,냉난방가전
2,벽걸이형 선풍기,1.1430,-0.1848,NaN,0.6333,0.5769,냉난방가전
77,업소용 선풍기,1.1012,-0.2628,-0.2232,0.5514,0.6307,냉난방가전
151,이동형 에어컨,0.8922,-0.2469,-0.1680,0.4979,0.4345,냉난방가전
107,탁상/USB 선풍기,0.7579,NaN,NaN,0.6083,0.5595,냉난방가전
79,냉풍기,0.7560,-0.2734,-0.1838,0.3671,0.3614,냉난방가전
120,휴대용 선풍기,0.7474,-0.2119,-0.0803,0.5811,0.5363,냉난방가전
5,벽걸이 에어컨,0.5293,-0.1803,0.1746,0.5276,0.3885,냉난방가전
61,전기온수기,0.4741,NaN,0.4917,0.6063,1.3891,냉난방가전
92,제습기,0.4048,0.3193,NaN,0.4690,0.4201,냉난방가전


In [524]:
warm.big('식품', '평균기온', 'plus').head(10)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
106,돼지 곱창,1.8227,0.2622,NaN,0.8029,0.2990,식품
101,참외/메론/수박,1.2937,NaN,NaN,0.6568,0.3297,식품
36,이온음료,1.2685,-0.1918,0.2676,0.8349,0.7286,식품
3,옥수수,1.1720,NaN,-0.2665,0.5887,0.3510,식품
54,다이어트용 헬스보충식품,1.1447,NaN,0.4697,0.8120,0.7439,식품
24,계란,1.1243,0.1895,0.1975,0.7487,0.8054,식품
74,김치류,1.1023,NaN,0.2301,0.7864,1.4760,식품
48,무김치,1.0896,NaN,0.2781,0.7723,1.0006,식품
9,배추김치,1.0784,0.1994,-0.1004,0.6932,1.2386,식품
38,딸기/복분자/블루베리,0.9528,NaN,0.7271,0.8374,0.7942,식품


### 음의 상관관계

In [423]:
warm.big('뷰티', '평균기온', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [424]:
warm.big('가전', '평균기온', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
94,공기청정기,-0.7419,0.3203,1.2679,0.7093,0.9641,냉난방가전
103,공기정화 용품,-0.2027,0.2361,0.9291,0.7144,1.1552,냉난방가전


In [425]:
warm.big('식품', '평균기온', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
41,포도/거봉/체리,-0.2166,0.1559,0.6589,0.5117,0.8671,식품


## 2.1.2 일강수량

### 양의 상관관계

In [659]:
warm.big('뷰티', '일강수량', 'plus').head(10)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
100,바디 세트,0.2836,0.1910,0.3647,0.6741,1.4192,뷰티
98,네일리무버,0.9144,0.1558,0.3814,0.8026,1.2453,뷰티
68,색조 메이크업 립밤,0.2559,0.1386,0.3973,0.6359,0.5543,뷰티
16,바디 클렌져,0.5797,0.1246,0.2864,0.7654,0.7974,뷰티
10,핸드크림,0.1826,0.1172,0.2770,0.5721,0.7471,뷰티
17,스킨케어 필오프팩,0.1721,0.1028,NaN,0.2415,1.8617,뷰티


In [665]:
rainy.big('식품', '일강수량', 'plus').head(10)[['sm_cat', '일강수량']]

,sm_cat,일강수량
32,낙지,0.4552


In [428]:
warm.big('식품', '일강수량', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
46,코코아/핫초코,0.8437,0.4727,0.9206,0.8161,0.8234,식품
90,율무차,0.3263,0.3619,0.9178,0.7208,0.9702,식품
122,감귤/한라봉/오렌지,NaN,0.3096,0.7744,0.5926,0.5622,식품
4,초코우유,NaN,0.2841,0.8369,0.7181,0.6779,식품
102,견과류 밤,0.4202,0.2760,0.4197,0.6421,0.9825,식품


In [668]:
whole.big('뷰티', '일강수량', 'plus').head(10) 

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
10,핸드크림,-0.2227,0.2719,0.6853,0.6403,0.5495,뷰티
68,색조 메이크업 립밤,-0.1557,0.1905,0.5738,0.6120,0.5136,뷰티
140,핸드워시/스크럽,0.8436,0.1766,0.5296,0.7334,0.6104,뷰티
8,바디케어용 때비누,NaN,0.1470,0.2820,0.5083,1.6715,뷰티
109,헤어스타일링용 흑채,0.1408,0.1253,0.4590,0.6909,1.1572,뷰티
130,기초 화장용 오일/앰플,0.1533,0.1222,0.3325,0.6069,0.9942,뷰티
80,네일관리 소품,0.6413,0.1192,0.5477,0.7624,1.0831,뷰티
16,바디 클렌져,0.5937,0.1056,0.2695,0.7600,0.7649,뷰티
115,뷰티용 면봉/귀이개,0.2092,0.1025,0.2295,0.6091,1.3320,뷰티
100,바디 세트,NaN,0.0814,0.2272,0.5481,1.5399,뷰티


### 음의 상관관계

In [429]:
warm.big('뷰티', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [527]:
warm.big('가전', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
79,냉풍기,0.7560,-0.2734,-0.1838,0.3671,0.3614,냉난방가전
77,업소용 선풍기,1.1012,-0.2628,-0.2232,0.5514,0.6307,냉난방가전
151,이동형 에어컨,0.8922,-0.2469,-0.1680,0.4979,0.4345,냉난방가전
120,휴대용 선풍기,0.7474,-0.2119,-0.0803,0.5811,0.5363,냉난방가전
34,에어컨 리모컨,1.1971,-0.1979,-0.2520,0.6181,0.4843,냉난방가전


In [431]:
warm.big('식품', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
36,이온음료,1.2685,-0.1918,0.2676,0.8349,0.7286,식품


## 2.1.3. 미세먼지

### 양의 상관관계

In [688]:
whole.big('뷰티', '평균기온', 'plus').head(10)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
19,바디케어용 제모제,1.1597,NaN,0.2520,0.7429,0.5378,뷰티
89,뷰티 타투,1.0996,NaN,0.0524,0.6988,0.8122,뷰티
142,애프터선,1.0780,NaN,0.1703,0.7386,0.8765,뷰티
97,선케어용 선밤,0.8507,-0.1482,0.1042,0.5373,0.5964,뷰티
25,스킨케어 코팩,0.8454,NaN,0.2670,0.7334,1.0962,뷰티
140,핸드워시/스크럽,0.8436,0.1766,0.5296,0.7334,0.6104,뷰티
29,데오드란트,0.7037,-0.0867,0.1688,0.6859,0.5558,뷰티
80,네일관리 소품,0.6413,0.1192,0.5477,0.7624,1.0831,뷰티
16,바디 클렌져,0.5937,0.1056,0.2695,0.7600,0.7649,뷰티
40,선크림,0.5857,NaN,0.2714,0.6696,0.5210,뷰티


In [433]:
warm.big('가전', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
94,공기청정기,-0.7419,0.3203,1.2679,0.7093,0.9641,냉난방가전
103,공기정화 용품,-0.2027,0.2361,0.9291,0.7144,1.1552,냉난방가전
51,복합식 가습기,NaN,NaN,0.6243,0.5547,0.7412,냉난방가전
61,전기온수기,0.4741,NaN,0.4917,0.6063,1.3891,냉난방가전
152,의류건조기,NaN,0.2765,0.4602,0.6150,1.4425,냉난방가전


In [434]:
warm.big('식품', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
26,둥굴레차,0.2454,0.2668,0.9211,0.7290,1.0518,식품
46,코코아/핫초코,0.8437,0.4727,0.9206,0.8161,0.8234,식품
90,율무차,0.3263,0.3619,0.9178,0.7208,0.9702,식품
65,유제품 음료,NaN,0.2701,0.8539,0.6920,0.5571,식품
4,초코우유,NaN,0.2841,0.8369,0.7181,0.6779,식품


### 음의 상관관계

In [435]:
warm.big('뷰티', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [436]:
warm.big('가전', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
34,에어컨 리모컨,1.1971,-0.1979,-0.2520,0.6181,0.4843,냉난방가전
77,업소용 선풍기,1.1012,-0.2628,-0.2232,0.5514,0.6307,냉난방가전
79,냉풍기,0.7560,-0.2734,-0.1838,0.3671,0.3614,냉난방가전
151,이동형 에어컨,0.8922,-0.2469,-0.1680,0.4979,0.4345,냉난방가전
120,휴대용 선풍기,0.7474,-0.2119,-0.0803,0.5811,0.5363,냉난방가전


In [437]:
warm.big('식품', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
3,옥수수,1.1720,NaN,-0.2665,0.5887,0.3510,식품
9,배추김치,1.0784,0.1994,-0.1004,0.6932,1.2386,식품


## 2.2. 시원한 달 (10 ~ 3월)

### 2.2.1. 평균기온

### 양의 상관관계

In [438]:
cool.big('뷰티', '평균기온', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
16,바디 클렌져,1.4441,NaN,0.7490,0.7362,0.7071,뷰티
140,핸드워시/스크럽,1.3581,NaN,0.7393,0.7179,0.6689,뷰티
89,뷰티 타투,1.2876,NaN,0.1037,0.5332,0.5490,뷰티
29,데오드란트,0.8735,NaN,0.6153,0.7454,0.7557,뷰티
40,선크림,0.8716,NaN,0.7697,0.7110,0.6917,뷰티


In [525]:
cool.big('가전', '평균기온', 'plus').head(10)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
56,온수매트,1.8990,NaN,0.1000,0.5400,0.6024,냉난방가전
83,카페트매트,1.8607,NaN,NaN,0.5355,0.8233,냉난방가전
49,황토매트,1.8016,NaN,0.3857,0.5642,0.5924,냉난방가전
86,전기장판,1.1248,NaN,0.1031,0.4995,0.7007,냉난방가전
92,제습기,0.9648,0.5528,0.5019,0.7339,1.1960,냉난방가전
2,벽걸이형 선풍기,0.9024,NaN,0.2121,0.4808,1.3365,냉난방가전
151,이동형 에어컨,0.8660,NaN,0.3658,0.5161,1.5417,냉난방가전
77,업소용 선풍기,0.8166,NaN,0.1761,0.3759,1.6377,냉난방가전
107,탁상/USB 선풍기,0.5823,NaN,0.0833,0.3482,1.1608,냉난방가전
61,전기온수기,0.5595,NaN,0.3942,0.4623,0.6068,냉난방가전


In [440]:
cool.big('식품', '평균기온', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
24,계란,2.2256,NaN,0.7814,0.7604,0.5869,식품
74,김치류,1.8840,NaN,0.3822,0.7306,0.6702,식품
84,감/홍시,1.8767,NaN,0.2551,0.5541,0.4505,식품
9,배추김치,1.8104,NaN,0.1880,0.7010,1.1368,식품
123,곡물차,1.7857,NaN,0.5863,0.7144,0.6226,식품


### 음의 상관관계

In [441]:
cool.big('뷰티', '평균기온', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
116,뷰티용 뷰러,-0.5159,0.4502,0.9512,0.7516,0.9436,뷰티


In [442]:
cool.big('가전', '평균기온', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [443]:
whole.big('식품', '평균기온', 'minus').head(30)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
127,굴 생물,-0.4038,0.1416,0.4098,0.3833,0.3451,식품
122,감귤/한라봉/오렌지,-0.3862,0.2283,0.6075,0.6566,0.5533,식품
110,해초류,-0.1867,0.1201,0.2587,0.3896,1.3532,식품
82,유자차,-0.1848,0.2705,0.6637,0.5617,0.4989,식품
90,율무차,-0.1442,0.2462,0.6419,0.6374,0.7228,식품
46,코코아/핫초코,-0.0798,0.2292,0.5216,0.6827,0.7124,식품
21,랍스타,-0.0796,NaN,0.1195,0.1489,1.4973,식품


In [526]:
cool.big('식품', '평균기온', 'minus').head(30)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
38,딸기/복분자/블루베리,-1.4524,0.4873,1.1375,0.7332,0.7362,식품
65,유제품 음료,-0.7715,0.3691,0.4803,0.4027,0.2276,식품
110,해초류,-0.4620,0.1925,0.2879,0.3679,1.4523,식품
11,전복 생물,-0.2633,NaN,0.2658,0.3686,0.8558,식품
76,인삼/수삼/산삼,-0.2324,NaN,0.3956,0.5253,0.6907,식품


## 2.2.2 일강수량

### 양의 상관관계

In [445]:
cool.big('뷰티', '일강수량', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
116,뷰티용 뷰러,-0.5159,0.4502,0.9512,0.7516,0.9436,뷰티
80,네일관리 소품,NaN,0.2886,0.9118,0.7576,1.0585,뷰티
67,클렌징 티슈,0.4269,0.2361,0.4717,0.6880,0.9359,뷰티
8,바디케어용 때비누,NaN,0.2281,0.2782,0.4962,1.5571,뷰티
25,스킨케어 코팩,0.4482,0.1660,0.3266,0.6140,1.5377,뷰티


In [446]:
cool.big('가전', '일강수량', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
92,제습기,0.9648,0.5528,0.5019,0.7339,1.1960,냉난방가전
152,의류건조기,NaN,0.2215,0.5760,0.6696,1.4645,냉난방가전


In [447]:
cool.big('식품', '일강수량', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
38,딸기/복분자/블루베리,-1.4524,0.4873,1.1375,0.7332,0.7362,식품
104,시금치,0.7250,0.4856,1.0005,0.7464,0.8082,식품
126,식혜/수정과,NaN,0.4734,0.9520,0.7359,0.9313,식품
45,탄산음료,NaN,0.4627,0.8696,0.7172,0.4401,식품
22,음용 식초,1.1898,0.3778,0.6299,0.7122,0.8871,식품


### 음의 상관관계

In [448]:
cool.big('뷰티', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [449]:
cool.big('가전', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [450]:
cool.big('식품', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


## 2.2.3. 미세먼지

### 양의 상관관계

In [451]:
cool.big('뷰티', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
116,뷰티용 뷰러,-0.5159,0.4502,0.9512,0.7516,0.9436,뷰티
80,네일관리 소품,NaN,0.2886,0.9118,0.7576,1.0585,뷰티
40,선크림,0.8716,NaN,0.7697,0.7110,0.6917,뷰티
16,바디 클렌져,1.4441,NaN,0.7490,0.7362,0.7071,뷰티
140,핸드워시/스크럽,1.3581,NaN,0.7393,0.7179,0.6689,뷰티


In [452]:
cool.big('가전', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
64,초음파식 가습기,NaN,NaN,0.7079,0.5150,0.3720,냉난방가전
152,의류건조기,NaN,0.2215,0.5760,0.6696,1.4645,냉난방가전
51,복합식 가습기,NaN,NaN,0.5253,0.4909,0.6948,냉난방가전
92,제습기,0.9648,0.5528,0.5019,0.7339,1.1960,냉난방가전
59,스탠드형 냉온풍기,NaN,NaN,0.4300,0.5967,1.0585,냉난방가전


In [453]:
cool.big('식품', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
38,딸기/복분자/블루베리,-1.4524,0.4873,1.1375,0.7332,0.7362,식품
104,시금치,0.7250,0.4856,1.0005,0.7464,0.8082,식품
6,생수,1.1924,NaN,0.9902,0.8014,0.7030,식품
30,파/양파,1.2972,0.3332,0.9768,0.7699,0.5870,식품
126,식혜/수정과,NaN,0.4734,0.9520,0.7359,0.9313,식품


### 음의 상관관계

In [454]:
cool.big('뷰티', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [455]:
cool.big('가전', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [456]:
cool.big('식품', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


# 3. 장마철

## 3.1. 평균기온

### 양의 상관관계

In [457]:
# rainy.big('뷰티', '평균기온', 'plus').head(5)

In [458]:
# rainy.big('가전', '평균기온', 'plus').head(5)

In [459]:
# rainy.big('식품', '평균기온', 'plus').head(5)

###  음의 상관관계

In [460]:
# rainy.big('뷰티', '평균기온', 'minus').head(5)

In [461]:
# rainy.big('가전', '평균기온', 'minus').head(5)

In [462]:
# rainy.big('식품', '평균기온', 'minus').head(5)

## 1.2. 일강수량

### 양의 상관관계

In [463]:
rainy.big('뷰티', '일강수량', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
100,바디 세트,NaN,0.4219,NaN,0.5942,1.7037,뷰티
97,선케어용 선밤,NaN,0.3854,NaN,0.7878,1.8410,뷰티


In [464]:
rainy.big('가전', '일강수량', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
92,제습기,1.3429,0.6351,NaN,0.6895,0.8371,냉난방가전
103,공기정화 용품,NaN,0.4765,1.2144,0.8657,1.3832,냉난방가전
143,온풍기,NaN,0.3534,NaN,0.5106,2.0147,냉난방가전


In [465]:
rainy.big('식품', '일강수량', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
32,낙지,NaN,0.4552,NaN,0.3224,1.9181,식품


### 음의 상관관계

In [466]:
rainy.big('뷰티', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [467]:
rainy.big('가전', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [468]:
rainy.big('식품', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
24,계란,2.6169,-0.4460,-1.1117,0.6685,1.3697,식품


## 3.3. 미세먼지

### 양의 상관관계

In [469]:
# rainy.big('뷰티', '미세먼지', 'plus').head(5)

In [470]:
# rainy.big('가전', '미세먼지', 'plus').head(5)

In [471]:
# rainy.big('식품', '미세먼지', 'plus').head(5)

### 음의 상관관계

In [472]:
# rainy.big('뷰티', '미세먼지', 'minus').head(5)

In [473]:
# rainy.big('가전', '미세먼지', 'minus').head(5)

In [474]:
# rainy.big('식품', '미세먼지', 'minus').head(5)

# 4. 미세먼지 고농도 기간

## 4.3. 미세먼지

### 양의 상관관계

In [475]:
dusty.big('뷰티', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
85,네일관리 소품,NaN,1.3758,0.9541,0.7602,1.1239,뷰티
124,뷰티용 뷰러,-1.1740,1.5444,0.9207,0.7549,1.1710,뷰티
150,핸드워시/스크럽,NaN,1.2512,0.8606,0.7197,0.7654,뷰티
10,핸드크림,-1.6764,1.2352,0.8430,0.6116,0.6285,뷰티
16,바디 클렌져,NaN,1.3834,0.8365,0.7207,0.8420,뷰티


In [476]:
dusty.big('가전', '미세먼지', 'plus').head(10)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
67,초음파식 가습기,-2.3774,1.1087,0.7391,0.5338,0.4141,냉난방가전
52,황토매트,-1.8682,1.7777,0.6998,0.5987,0.8238,냉난방가전
99,제습기,1.2947,1.1367,0.6060,0.7183,1.2336,냉난방가전
92,전기장판,-1.5904,0.9155,0.5618,0.4831,0.4081,냉난방가전
163,의류건조기,NaN,0.7327,0.5539,0.6616,1.6084,냉난방가전
54,복합식 가습기,-1.5023,0.7856,0.5359,0.5114,0.8274,냉난방가전
156,에어워셔,-0.7243,0.6957,0.5127,0.6368,1.0117,냉난방가전
64,전기온수기,-0.6513,0.9363,0.4706,0.4505,0.7281,냉난방가전
88,카페트매트,-1.2264,0.7829,0.4597,0.4145,0.6255,냉난방가전
62,스탠드형 냉온풍기,NaN,0.6450,0.4353,0.5815,1.0416,냉난방가전


In [477]:
dusty.big('식품', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
40,딸기/복분자/블루베리,-1.8585,1.7483,1.1581,0.7704,0.8756,식품
111,시금치,-0.9933,1.5533,1.0932,0.7523,0.8884,식품
31,파/양파,NaN,1.5668,1.0662,0.7694,0.6579,식품
6,생수,NaN,1.2905,1.0583,0.7864,0.7085,식품
143,고추/피망/파프리카,NaN,1.4825,0.9638,0.7288,0.6700,식품


### 음의 상관관계

In [478]:
dusty.big('뷰티', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [479]:
dusty.big('가전', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [480]:
dusty.big('식품', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


#  4. 2018 / 2019

## 4.1. 2018

### 4.1.1. 평균기온

### 양의 상관관계

In [604]:
year2018.big('뷰티', '평균기온', 'minus').head(5)[['sm_cat', '평균기온']]

,sm_cat,평균기온
10,핸드크림,-0.5509
68,색조 메이크업 립밤,-0.3003


In [605]:
year2018.big('가전', '평균기온', 'minus').head(5)[['sm_cat', '평균기온']]

,sm_cat,평균기온
64,초음파식 가습기,-0.7183
44,가열식 가습기,-0.5886
51,복합식 가습기,-0.5248
150,컨벡터,-0.3897
50,히터,-0.3684


In [606]:
year2018.big('식품', '평균기온', 'minus').head(5)[['sm_cat', '평균기온']]

,sm_cat,평균기온
122,감귤/한라봉/오렌지,-0.6817
127,굴 생물,-0.6293
46,코코아/핫초코,-0.4583
104,시금치,-0.4205
65,유제품 음료,-0.4018


In [607]:
year2019.big('뷰티', '평균기온', 'minus').head(5)[['sm_cat', '평균기온']]

,sm_cat,평균기온


In [608]:
year2019.big('가전', '평균기온', 'minus').head(5)[['sm_cat', '평균기온']]

,sm_cat,평균기온
64,초음파식 가습기,-0.5581
51,복합식 가습기,-0.3669
150,컨벡터,-0.3034
50,히터,-0.2820
143,온풍기,-0.2798


In [609]:
year2019.big('식품', '평균기온', 'minus').head(5)[['sm_cat', '평균기온']]

,sm_cat,평균기온
127,굴 생물,-0.3926
122,감귤/한라봉/오렌지,-0.3364
110,해초류,-0.1646
21,랍스타,-0.1138


In [616]:
year2018.power('일강수량', 'plus').head(10)[['일강수량']]

,일강수량
둥굴레차,0.5787
코코아/핫초코,0.5694
시금치,0.5419
소고기 육회,0.5037
곡물차,0.4601
돼지 곱창,0.4541
딸기/복분자/블루베리,0.4528
가공란,0.4216
계란,0.4149
핸드크림,0.4136


In [615]:
year2019.power('일강수량', 'plus').head(10)[['일강수량']]

,일강수량
돼지 곱창,0.5050
꽃게,0.2939
콜라겐/코큐텐 영양제,0.2765
생닭/닭부분육,0.2670
조개,0.2640
핸드크림,0.2424
감귤/한라봉/오렌지,0.2240
황토매트,0.2173
견과류 밤,0.2088
수산 생물,0.1993


### 음의 상관관계

In [484]:
year2018.big('뷰티', '평균기온', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
10,핸드크림,-0.5509,0.4136,0.9973,0.6598,0.5427,뷰티
68,색조 메이크업 립밤,-0.3003,0.2189,0.5625,0.5902,0.5464,뷰티


In [485]:
year2018.big('가전', '평균기온', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
64,초음파식 가습기,-0.7183,0.2374,0.7559,0.4812,0.3510,냉난방가전
44,가열식 가습기,-0.5886,NaN,0.6556,0.4215,0.4141,냉난방가전
51,복합식 가습기,-0.5248,0.1759,0.5669,0.4566,0.7270,냉난방가전
150,컨벡터,-0.3897,NaN,0.3809,0.2713,0.4811,냉난방가전
50,히터,-0.3684,NaN,0.3770,0.3159,0.4053,냉난방가전


In [486]:
year2018.big('식품', '평균기온', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
122,감귤/한라봉/오렌지,-0.6817,0.3344,0.8345,0.6185,0.6061,식품
127,굴 생물,-0.6293,0.2324,0.5829,0.5267,0.4816,식품
46,코코아/핫초코,-0.4583,0.5694,1.0604,0.7082,0.6891,식품
104,시금치,-0.4205,0.5419,1.0970,0.7519,0.9049,식품
65,유제품 음료,-0.4018,0.3480,0.7456,0.5854,0.3837,식품


## 4.1.2 일강수량

### 양의 상관관계

In [598]:
year2018.big('뷰티', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
97,선케어용 선밤,0.9188,-0.1743,0.1531,0.5353,0.6209,뷰티


In [599]:
year2018.big('가전', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
79,냉풍기,0.7832,-0.2817,-0.0796,0.3753,0.3609,냉난방가전
5,벽걸이 에어컨,0.7917,-0.2406,NaN,0.4857,0.2999,냉난방가전
77,업소용 선풍기,0.9605,-0.2390,-0.0899,0.4919,0.3728,냉난방가전
151,이동형 에어컨,0.7476,-0.2383,-0.0617,0.4076,0.2944,냉난방가전
111,멀티형 에어컨,0.5790,-0.2337,NaN,0.3489,1.1233,냉난방가전


In [600]:
year2018.big('식품', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
101,참외/메론/수박,1.4993,-0.1934,NaN,0.6612,0.2790,식품


In [601]:
year2019.big('뷰티', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
29,데오드란트,0.8131,-0.1614,0.1709,0.6169,0.5082,뷰티
142,애프터선,1.1196,-0.1456,0.1739,0.7535,0.8483,뷰티


In [602]:
year2019.big('가전', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
2,벽걸이형 선풍기,1.2385,-0.1686,NaN,0.5835,0.5283,냉난방가전
107,탁상/USB 선풍기,0.8590,-0.1339,NaN,0.5372,0.5658,냉난방가전
59,스탠드형 냉온풍기,0.3626,-0.0918,0.1373,0.5655,1.1305,냉난방가전


In [603]:
year2019.big('식품', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
36,이온음료,1.4309,-0.1848,0.4064,0.8277,0.7176,식품


### 음의 상관관계

In [490]:
year2018.big('뷰티', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
97,선케어용 선밤,0.9188,-0.1743,0.1531,0.5353,0.6209,뷰티


In [491]:
year2018.big('가전', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
79,냉풍기,0.7832,-0.2817,-0.0796,0.3753,0.3609,냉난방가전
5,벽걸이 에어컨,0.7917,-0.2406,NaN,0.4857,0.2999,냉난방가전
77,업소용 선풍기,0.9605,-0.2390,-0.0899,0.4919,0.3728,냉난방가전
151,이동형 에어컨,0.7476,-0.2383,-0.0617,0.4076,0.2944,냉난방가전
111,멀티형 에어컨,0.5790,-0.2337,NaN,0.3489,1.1233,냉난방가전


In [492]:
year2018.big('식품', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
101,참외/메론/수박,1.4993,-0.1934,NaN,0.6612,0.2790,식품


## 4.1.3. 미세먼지

### 양의 상관관계

In [493]:
year2018.big('뷰티', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
10,핸드크림,-0.5509,0.4136,0.9973,0.6598,0.5427,뷰티
140,핸드워시/스크럽,0.4704,0.4100,0.6531,0.7332,0.8630,뷰티
80,네일관리 소품,0.5040,0.3160,0.5923,0.7552,1.2331,뷰티
16,바디 클렌져,0.8665,0.3146,0.5770,0.7785,0.9107,뷰티
68,색조 메이크업 립밤,-0.3003,0.2189,0.5625,0.5902,0.5464,뷰티


In [494]:
year2018.big('가전', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
64,초음파식 가습기,-0.7183,0.2374,0.7559,0.4812,0.3510,냉난방가전
44,가열식 가습기,-0.5886,NaN,0.6556,0.4215,0.4141,냉난방가전
61,전기온수기,-0.2387,0.3044,0.6530,0.5190,0.8808,냉난방가전
51,복합식 가습기,-0.5248,0.1759,0.5669,0.4566,0.7270,냉난방가전
49,황토매트,NaN,0.2481,0.5567,0.3470,0.3072,냉난방가전


In [495]:
year2018.big('식품', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
26,둥굴레차,NaN,0.5787,1.1159,0.7077,0.9361,식품
104,시금치,-0.4205,0.5419,1.0970,0.7519,0.9049,식품
46,코코아/핫초코,-0.4583,0.5694,1.0604,0.7082,0.6891,식품
38,딸기/복분자/블루베리,NaN,0.4528,1.0533,0.7167,0.5840,식품
123,곡물차,NaN,0.4601,0.8354,0.6287,0.5935,식품


### 음의 상관관계

In [496]:
year2018.big('뷰티', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [497]:
year2018.big('가전', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
77,업소용 선풍기,0.9605,-0.2390,-0.0899,0.4919,0.3728,냉난방가전
79,냉풍기,0.7832,-0.2817,-0.0796,0.3753,0.3609,냉난방가전
34,에어컨 리모컨,0.9274,-0.2077,-0.0716,0.5300,0.3564,냉난방가전
151,이동형 에어컨,0.7476,-0.2383,-0.0617,0.4076,0.2944,냉난방가전
120,휴대용 선풍기,0.7724,-0.2300,-0.0459,0.5412,0.5500,냉난방가전


In [498]:
year2018.big('식품', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


## 4.2. 2019년

### 4.2.1. 평균기온

### 양의 상관관계

In [591]:
year2019.big('뷰티', '평균기온', 'plus').head(5)[['sm_cat', '평균기온']]

,sm_cat,평균기온
140,핸드워시/스크럽,1.3747
108,네일컬러,1.3452
89,뷰티 타투,1.3218
19,바디케어용 제모제,1.2681
58,네일 메이크업 용품,1.1703


In [592]:
year2019.big('가전', '평균기온', 'plus').head(5)[['sm_cat', '평균기온']]

,sm_cat,평균기온
120,휴대용 선풍기,1.4936
34,에어컨 리모컨,1.2438
2,벽걸이형 선풍기,1.2385
79,냉풍기,1.0744
151,이동형 에어컨,0.9173


In [593]:
year2019.big('식품', '평균기온', 'plus').head(5)[['sm_cat', '평균기온']]

,sm_cat,평균기온
101,참외/메론/수박,1.6677
54,다이어트용 헬스보충식품,1.5078
36,이온음료,1.4309
74,김치류,1.3516
87,탄산수,1.3035


In [597]:
year2018.power('평균기온', 'neutral').head(5)

,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson
아이스티,1.5966,NaN,0.3704,0.8105,0.5572
미숫가루/곡물가루,1.5022,NaN,0.2075,0.8187,1.0541
참외/메론/수박,1.4993,-0.1934,NaN,0.6612,0.2790
오이/가지,1.3556,0.3605,0.7136,0.8435,1.0110
이온음료,1.3432,NaN,0.2268,0.7994,0.6842


In [596]:
year2019.power('평균기온', 'neutral').head(5)

,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson
참외/메론/수박,1.6677,NaN,NaN,0.6548,0.3857
다이어트용 헬스보충식품,1.5078,NaN,0.4262,0.8241,0.9307
휴대용 선풍기,1.4936,NaN,-0.1310,0.5812,0.3924
이온음료,1.4309,-0.1848,0.4064,0.8277,0.7176
핸드워시/스크럽,1.3747,NaN,0.5019,0.8108,0.8965


### 음의 상관관계

In [502]:
year2019.big('뷰티', '평균기온', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [503]:
year2019.big('가전', '평균기온', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
64,초음파식 가습기,-0.5581,NaN,0.7082,0.4474,0.3256,냉난방가전
51,복합식 가습기,-0.3669,NaN,0.5587,0.4550,0.5269,냉난방가전
150,컨벡터,-0.3034,NaN,0.3829,0.2408,0.5094,냉난방가전
50,히터,-0.2820,NaN,0.3748,0.2359,0.3057,냉난방가전
143,온풍기,-0.2798,NaN,0.3778,0.2481,0.3216,냉난방가전


In [504]:
year2019.big('식품', '평균기온', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
127,굴 생물,-0.3926,NaN,0.4352,0.3280,0.3102,식품
122,감귤/한라봉/오렌지,-0.3364,0.2240,0.6174,0.6796,0.5412,식품
110,해초류,-0.1646,NaN,0.3311,0.3648,1.3204,식품
21,랍스타,-0.1138,NaN,0.1546,0.1752,1.6539,식품


## 4.2.2 일강수량

### 양의 상관관계

In [505]:
year2019.big('뷰티', '일강수량', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
10,핸드크림,NaN,0.2424,0.6398,0.6137,0.5777,뷰티
85,풋패치,0.4097,0.1178,0.2623,0.6872,1.1667,뷰티
115,뷰티용 면봉/귀이개,0.3144,0.1124,0.2117,0.5955,1.4847,뷰티


In [506]:
year2019.big('가전', '일강수량', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
49,황토매트,NaN,0.2173,0.4296,0.4092,0.5277,냉난방가전


In [507]:
year2019.big('식품', '일강수량', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
106,돼지 곱창,1.1289,0.5050,0.6931,0.7705,0.6474,식품
125,꽃게,0.3286,0.2939,0.0872,0.3710,1.4607,식품
118,콜라겐/코큐텐 영양제,0.6864,0.2765,0.3646,0.6449,1.3513,식품
138,생닭/닭부분육,0.7025,0.2670,0.3546,0.7607,0.6937,식품
72,조개,0.1252,0.2640,0.5013,0.6959,1.3566,식품


### 음의 상관관계

In [508]:
year2019.big('뷰티', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
29,데오드란트,0.8131,-0.1614,0.1709,0.6169,0.5082,뷰티
142,애프터선,1.1196,-0.1456,0.1739,0.7535,0.8483,뷰티


In [509]:
year2019.big('가전', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
2,벽걸이형 선풍기,1.2385,-0.1686,NaN,0.5835,0.5283,냉난방가전
107,탁상/USB 선풍기,0.8590,-0.1339,NaN,0.5372,0.5658,냉난방가전
59,스탠드형 냉온풍기,0.3626,-0.0918,0.1373,0.5655,1.1305,냉난방가전


In [510]:
year2019.big('식품', '일강수량', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
36,이온음료,1.4309,-0.1848,0.4064,0.8277,0.7176,식품


## 4.2.3. 미세먼지

### 양의 상관관계

In [511]:
year2019.big('뷰티', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
68,색조 메이크업 립밤,NaN,NaN,0.8293,0.6328,0.5116,뷰티
109,헤어스타일링용 흑채,0.3530,NaN,0.6846,0.7211,1.1986,뷰티
10,핸드크림,NaN,0.2424,0.6398,0.6137,0.5777,뷰티
80,네일관리 소품,0.8360,NaN,0.5367,0.7658,1.1393,뷰티
140,핸드워시/스크럽,1.3747,NaN,0.5019,0.8108,0.8965,뷰티


In [512]:
year2019.big('가전', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
64,초음파식 가습기,-0.5581,NaN,0.7082,0.4474,0.3256,냉난방가전
51,복합식 가습기,-0.3669,NaN,0.5587,0.4550,0.5269,냉난방가전
152,의류건조기,NaN,NaN,0.5337,0.6833,1.2743,냉난방가전
49,황토매트,NaN,0.2173,0.4296,0.4092,0.5277,냉난방가전
61,전기온수기,NaN,NaN,0.4200,0.4155,0.5667,냉난방가전


In [513]:
year2019.big('식품', '미세먼지', 'plus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
38,딸기/복분자/블루베리,NaN,NaN,0.9825,0.7376,0.6840,식품
90,율무차,NaN,NaN,0.9038,0.6367,0.7017,식품
26,둥굴레차,NaN,NaN,0.8911,0.6988,0.9688,식품
104,시금치,0.2729,NaN,0.7705,0.7370,0.7583,식품
123,곡물차,0.5607,NaN,0.6978,0.6450,0.4377,식품


### 음의 상관관계

In [514]:
year2019.big('뷰티', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat


In [515]:
year2019.big('가전', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
120,휴대용 선풍기,1.4936,NaN,-0.1310,0.5812,0.3924,냉난방가전
34,에어컨 리모컨,1.2438,NaN,-0.0947,0.5738,0.4848,냉난방가전
79,냉풍기,1.0744,NaN,-0.0913,0.4958,0.5081,냉난방가전
77,업소용 선풍기,0.7180,NaN,-0.0708,0.5025,0.9082,냉난방가전
151,이동형 에어컨,0.9173,NaN,-0.0670,0.5240,0.6207,냉난방가전


In [516]:
year2019.big('식품', '미세먼지', 'minus').head(5)

,sm_cat,평균기온,일강수량,미세먼지,adjusted R squared,Durbin-Watson,big_cat
3,옥수수,1.0630,NaN,-0.0833,0.5460,0.3398,식품
